In [291]:
import numpy as np
import pandas as pd

In [292]:
선행지수 = pd.read_csv("./_경기선행지수.csv")
건설업주가지수 = pd.read_csv("./_건설업주가지수.csv")
국고채 = pd.read_csv("./_국고채(3년).csv")
GDP = pd.read_csv("./_명목GDP_실질GDP변화율.csv")
산업생산지수 = pd.read_csv("./_산업생산지수.csv")
소비자물가지수 = pd.read_csv("./_소비자물가지수.csv")
코스피 = pd.read_csv("./_코스피지수.csv")
환율 = pd.read_csv("./_환율.csv")

* 선행지수, 건설업주가지수, 소비자물가지수, 산업생산지수, 환율

In [293]:
선행지수['날짜'] = pd.to_datetime(선행지수['날짜'], format='%Y.%m')
선행지수['날짜'] =  pd.to_datetime(선행지수['날짜'].dt.strftime('%Y-%m'))
#선행지수 = 선행지수.set_index('날짜')

In [294]:
건설업주가지수['날짜'] = pd.to_datetime(건설업주가지수['날짜'].apply(lambda x: f"{int(x):04d}-{int((x * 100) % 100):02d}"))

In [295]:
소비자물가지수['날짜'] =  pd.to_datetime(소비자물가지수['날짜'].apply(lambda x: f"{int(x):04d}-{int((x * 100) % 100):02d}"))

In [296]:
산업생산지수['날짜'][산업생산지수['날짜'] == '2023.11 p)'] = '2023.11'
산업생산지수['날짜'][산업생산지수['날짜'] == '2023.12 p)'] = '2023.12'

산업생산지수['날짜'] = pd.to_datetime(산업생산지수['날짜'])
산업생산지수['날짜'] = pd.to_datetime(산업생산지수['날짜'].dt.strftime('%Y-%m'))

#산업생산지수 = 산업생산지수.set_index('날짜')

<ipython-input-296-58392a65a3d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  산업생산지수['날짜'][산업생산지수['날짜'] == '2023.11 p)'] = '2023.11'
<ipython-input-296-58392a65a3d2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  산업생산지수['날짜'][산업생산지수['날짜'] == '2023.12 p)'] = '2023.12'


In [297]:
환율['날짜'] = pd.to_datetime(pd.to_datetime(환율['날짜']).dt.strftime('%Y-%m'))
#환율 = 환율.set_index('날짜')

* 국고채 전처리

In [298]:
# '월'제거 후 날짜데이터타입으로 변경
국고채['날짜'] = pd.to_datetime(국고채['날짜'].str.replace('월', ''), format='%Y%m')

# 중복행 제거
국고채 = 국고채.drop_duplicates().reset_index(drop=True)
국고채['날짜'] =  pd.to_datetime(국고채['날짜'].dt.strftime('%Y-%m'))
#국고채 = 국고채.set_index('날짜')

* GDP 전처리

In [299]:
# ','제거 후 수치형으로 변경
GDP['국내총생산(명목GDP)'] = GDP['국내총생산(명목GDP)'].str.replace(',', '').astype(float)

# 분기별 값은 해당 분기에 해당하는 월별 값으로 설정하는 함수 정의
def 분기to월 (row):
    year_분기 = row['날짜']
    year = int(year_분기[:4]) #year만 가져옴
    분기 = int(year_분기[4]) #분기만 가져옴
    months_in_분기 = [(year, 분기 * 3 - 2), (year, 분기 * 3 - 1), (year, 분기 * 3)] #해당 분기에 해당하는 월 list
    return pd.DataFrame({
        '날짜': [f"{year}{month:02d}" for year, month in months_in_분기],
        '국내총생산(명목GDP)': [row['국내총생산(명목GDP)']] * 3,
        '경제성장률(실질GDP성장률)': [row['경제성장률(실질GDP성장률)']] * 3
    })

gdp_new = pd.concat([분기to월(row) for index, row in GDP.iterrows()])

gdp_new.reset_index(drop=True, inplace=True)
gdp_new['날짜'] = pd.to_datetime(gdp_new['날짜'], format='%Y%m')

gdp_new['날짜'] = pd.to_datetime(gdp_new['날짜'].dt.strftime('%Y-%m'))
#gdp_new = gdp_new.set_index('날짜')

* 코스피

In [300]:
코스피['종가'] = 코스피['종가'].str.replace(',', '').astype(float)
코스피['날짜'] = pd.to_datetime(코스피['날짜'])
코스피['날짜'] = pd.to_datetime(코스피['날짜'].dt.strftime('%Y-%m'))
코스피 = 코스피.sort_values('날짜').reset_index(drop = True)

변수 데이터 합치기

In [301]:
var = [선행지수, 건설업주가지수, 소비자물가지수, 산업생산지수, 환율, 국고채, 코스피, gdp_new]
var_df = pd.concat(var, axis=1).drop(columns='날짜').set_index(선행지수['날짜'])

In [303]:
var_df.to_csv("./전처리완료변수.csv")